In [2]:
import cv2
import numpy as np
import math
cap = cv2.VideoCapture(0)
while(cap.isOpened()):
    ret, img = cap.read()
    crop_img = img[0:300, 0:300]
    grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
    value = (5, 5)
    blurred = cv2.GaussianBlur(grey, value, 15)
    _, thresh1 = cv2.threshold(blurred, 127, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    img,contours, hierarchy = cv2.findContours(thresh1.copy(),cv2.RETR_TREE, \
            cv2.CHAIN_APPROX_NONE)
    max_area = -1
    for i in range(len(contours)):  #trazimo najvecu konturu
        cnt=contours[i]
        area = cv2.contourArea(cnt)
        if(area>max_area):
            max_area=area
            ci=i
    cnt=contours[ci]
    x,y,w,h = cv2.boundingRect(cnt)   #uzimamo koordinate, duzinu i sirinu najvece konture
    #cv2.rectangle(crop_img,(x,y),(x+w,y+h),(0,0,255),0) #crtamo pravougaonik oko konture
    hull = cv2.convexHull(cnt)  #vraca konveksan oblik sake
    drawing = np.zeros(crop_img.shape,np.uint8)
    cv2.drawContours(drawing,[cnt],0,(0,255,0),0)  #crta saku kakva jeste
    cv2.drawContours(drawing,[hull],0,(0,0,255),0) # crta konveksnu sa
    hull = cv2.convexHull(cnt,returnPoints = False)
    defects = cv2.convexityDefects(cnt,hull)
    count_defects = 0
    #cv2.drawContours(thresh1, contours, -1, (0,255,0), 3)
    for i in range(defects.shape[0]):
        s,e,f,d = defects[i,0]
        #s-pocetak defekta
        #e-kraj defekta
        #f-najdalja tacka defekta
        #udaljenost izmedju f i konveksnog mnogougla
        start = tuple(cnt[s][0])
        end = tuple(cnt[e][0])
        far = tuple(cnt[f][0])
        a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
        b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
        c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
        angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
        if angle <= 90:
            count_defects += 1
            #cv2.circle(crop_img,far,1,[255,255,255],-1)
            cv2.circle(drawing, far, 2, [255,255,255],-1)
        #cv2.line(crop_img,start,end,[0,255,0],2)
    all_img = np.hstack((crop_img,drawing))
    cv2.imshow('Ruka ruku mije', all_img)
    cv2.imshow('Threshold', thresh1 )
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()
